In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_risk_df = pd.read_csv("Resources/lending_data.csv")

# Review the DataFrame
lending_risk_df.sample(5)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
45780,8100.0,6.561,42300,0.290780,2,0,12300,0
12496,6900.0,6.044,37500,0.200000,1,0,7500,0
59497,8600.0,6.759,44200,0.321267,3,0,14200,0
49639,10000.0,7.357,49800,0.397590,4,0,19800,0
54073,10600.0,7.645,52500,0.428571,5,1,22500,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_risk_df['loan_status']

# Separate the X variable, the features
x = lending_risk_df.drop(columns= 'loan_status')

In [4]:
# Review the y variable Series
y.sample(5)

18147    0
30782    0
52400    0
66107    0
31381    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
x.sample(5)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
9431,7700.0,6.390,40700,0.262899,2,0,10700
57153,9300.0,7.094,47400,0.367089,3,0,17400
2855,10000.0,7.359,49900,0.398798,4,0,19900
40965,9900.0,7.349,49800,0.397590,4,0,19800
36671,7700.0,6.392,40700,0.262899,2,0,10700


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 1, stratify= y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model1 = LogisticRegression(random_state = 1)

# Fit the model using training data
model1.fit(x_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
y_pred = model1.predict(x_test)

#Data comparison
results = pd.DataFrame({'prediction': y_pred, 'real': y_test}).reset_index(drop = True)
results.head()

,prediction,real
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, y_pred)

0.9442676901753825

In [11]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_pred, y_test))

[[18679    67]
 [   80   558]]


In [12]:
# Print the classification report for the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Overall the model is accurate, with a precision of 94%. With it being able to tell healthy loans 100% of the time but high risk loans 87% of the time. Although, paired with the recall rate (that is the correct identification of type of loan) for high risk loans is 89% the model is overal satisfactory on its high risk loan side of things but a better model should still be considered.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [13]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversampler_model = RandomOverSampler(random_state = 1)

# Fit the original training data to the random_oversampler model
x_resample, y_resample = random_oversampler_model.fit_resample(x_train, y_train)

In [14]:
# Count the distinct values of the resampled labels data
y_resample.value_counts()

loan_status
0    56277
1    56277
Name: count, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [15]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model2 = LogisticRegression(random_state = 1)

# Fit the model using the resampled training data
model2.fit(x_resample, y_resample)

# Make a prediction using the testing data
y_pred_1 = model2.predict(x_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [16]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, y_pred_1)

0.9959744975744975

In [17]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, y_pred_1))

[[18668    91]
 [    2   623]]


In [18]:
# Print the classification report for the model
print(classification_report(y_test, y_pred_1))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** There is a slight improvment from the original regression model. Notably in the accuracy score, going from around 94% to 99%. There is now 100% in the recall ability for both healthy and high risk loans, where that wasnt the case with the original model. Overall this model scores greater than the previous model, making it the better model for use. 